# Data Manipulation Efficiency in Pandas  

In this notebook, I am comparing the efficiency of different ways of manipulating data in Pandas using the credit default dataset.  

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

['UCI_Credit_Card.csv']


In [2]:
df = pd.read_csv("../input/UCI_Credit_Card.csv")

In [3]:
df.shape

(30000, 25)

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')

### Comparing apply, map, replace for ONE column

In [5]:
# Separating status information out from pay columns
def map_status(x):
    if x == -2:
        # No consumption
        return(0)
    elif x == -1:
        # Full pay
        return(1)
    elif x == 0:
        # Revolving credit
        return(2)
    elif x > 0:
        return(3)
    
map_status_dict = {-2: 0, -1: 1, 0: 2, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3}

In [6]:
%%timeit
df["PAY_0_STATUS"] = df["PAY_0"].apply(map_status)

27.7 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
df["PAY_0_STATUS"] = df["PAY_0"].map(map_status_dict)

1.48 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
df["PAY_0_STATUS"] = df["PAY_0"].replace(map_status_dict)

2.77 ms ± 392 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
df["PAY_0_STATUS"] = df.loc[:, "PAY_0"]*(df.loc[:, "PAY_0"] <= 0)+2

1.62 ms ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit
for index, row in df.iterrows():
    row["PAY_0_STATUS"] = map_status_dict[row["PAY_0"]]

2.06 s ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Speed: map > replace >> apply >>> looping with iterrows()  

### Comparing applymap & replace for MULTIPLE columns  

In [11]:
%%timeit
df.loc[:, "PAY_0":"PAY_6"].replace(map_status_dict)

7.14 ms ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
df.loc[:, "PAY_0":"PAY_6"].applymap(map_status_dict.get)

157 ms ± 755 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
(df.loc[:, "PAY_0":"PAY_6"] - 1)*(df.loc[:, "PAY_0":"PAY_6"] <= 0) +3

3.49 ms ± 22.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
((df.loc[:, "PAY_0":"PAY_6"] - 1)*(df.loc[:, "PAY_0":"PAY_6"] <= 0) +3).head()

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
0,3,3,1,1,0,0
1,1,3,2,2,2,3
2,2,2,2,2,2,2
3,2,2,2,2,2,2
4,1,2,1,2,2,2


In [15]:
(df.loc[:, "PAY_0":"PAY_6"].replace(map_status_dict)).head()

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
0,3,3,1,1,0,0
1,1,3,2,2,2,3
2,2,2,2,2,2,2
3,2,2,2,2,2,2
4,1,2,1,2,2,2


#### Speed: vectorization > replace >> applymap  

## Comparing vectorization speeds  

Lets compare the speed at which the square of each type of array are computed.  

In [16]:
np_rand_array = np.random.randint(1, 500, 10000)
pd_rand_series = pd.Series(np_rand_array)

#### 1. Apply() method on Pandas Series  

In [17]:
%%timeit
square_series1 = pd_rand_series.apply(lambda x: x**2)  

10.3 ms ± 59.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 2. Pandas inherent vectorization  

In [18]:
%%timeit
square_series2 = pd_rand_series**2

174 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### 3. Numpy inherent vectorization  

In [19]:
%%timeit
square_array = np_rand_array**2

6.63 µs ± 79 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Speed: numpy vectorization >> pandas vectorization >> pandas apply()  

In [20]:
# Delay months: only when PAY_0 > 0
df["pay0_delay"] = df["PAY_0"]*(df["PAY_0"] > 0)